In [1]:
%load_ext tensorboard

import shutil, os
if os.path.exists("logs"):
    shutil.rmtree("logs")


In [2]:

from datetime import datetime
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorboard.plugins.hparams import api as hp


print("TF Version:", tf.__version__)



TF Version: 2.17.0


In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0

x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]



In [4]:
HP_UNITS = hp.HParam('units', hp.Discrete([16, 32]))
HP_OPT = hp.HParam('opt', hp.Discrete(['adam', 'sgd']))
METRIC = 'accuracy'

with tf.summary.create_file_writer('logs/hparams').as_default():
    hp.hparams_config(
        hparams=[HP_UNITS, HP_OPT],
        metrics=[hp.Metric(METRIC, display_name='Accuracy')]
    )



In [5]:
def build_model(hparams):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28, 1)),
        keras.layers.Dense(hparams[HP_UNITS], activation='relu', name="embed_layer"),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=hparams[HP_OPT],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def run_model(run_dir, hparams):
    model = build_model(hparams)
    tb = keras.callbacks.TensorBoard(log_dir=run_dir, histogram_freq=1, profile_batch='5,10')
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        model.fit(x_train, y_train, epochs=1, batch_size=256, callbacks=[tb])
        loss, acc = model.evaluate(x_test, y_test)
        tf.summary.scalar(METRIC, acc, step=1)
    return model



In [6]:
models = []
session = 0

for u in HP_UNITS.domain.values:
    for opt in HP_OPT.domain.values:
        print(f"Running trial {session}")
        hparams = {HP_UNITS: u, HP_OPT: opt}
        run_dir = f"logs/hparams/run-{session}"
        model = run_model(run_dir, hparams)
        models.append(model)
        session += 1



Running trial 0


f:\MS in CS\MlOps\Labs\Lab5\Tensorboard\tf_env\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


235/235 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.7715 - loss: 0.8253
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9007 - loss: 0.3715
Running trial 1
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.3790 - loss: 1.9070
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5887 - loss: 1.4933
Running trial 2
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8305 - loss: 0.6391
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9093 - loss: 0.3245
Running trial 3
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.4650 - loss: 1.8130
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7424 - loss: 1.3012


In [8]:
from datetime import datetime
import tensorflow as tf

@tf.function
def custom_fn(x):
    return tf.nn.relu(x * 2)

# Reset any previously active traces
tf.summary.trace_off()

# Directory to save trace
trace_dir = f"logs/trace/{datetime.now().strftime('%H%M%S')}"
writer = tf.summary.create_file_writer(trace_dir)

# Start tracing with profiler enabled
tf.summary.trace_on(graph=True, profiler=True, profiler_outdir=trace_dir)

# Run your function to generate a trace
_ = custom_fn(tf.ones((3, 3)))

# Export the trace
with writer.as_default():
    tf.summary.trace_export(name="custom_fn_trace", step=0)


In [9]:
from tensorboard.plugins import projector

inputs = keras.Input(shape=(28, 28), name="input_layer")
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(16, activation='relu', name="embed_layer")(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

_ = model(np.random.rand(1, 28, 28))  # dummy call to initialize

embed_model = keras.Model(
    inputs=model.input,
    outputs=model.get_layer("embed_layer").output
)

sample_img = x_train[:50]  # use your dataset here
sample_lbl = y_train[:50]

embeddings = embed_model.predict(sample_img)

embed_dir = f"logs/embeddings/{datetime.now().strftime('%H%M%S')}"
os.makedirs(embed_dir, exist_ok=True)

np.savetxt(os.path.join(embed_dir, 'embeddings.tsv'), embeddings, delimiter='\t')
np.savetxt(os.path.join(embed_dir, 'metadata.tsv'), sample_lbl, fmt='%d')

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = 'embedding_tensor'
embedding.metadata_path = 'metadata.tsv'

projector.visualize_embeddings(embed_dir, config)

print(f"Embeddings saved for TensorBoard projector at: {embed_dir}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step 
Embeddings saved for TensorBoard projector at: logs/embeddings/225213


In [12]:
%load_ext tensorboard
%tensorboard --logdir logs --port 6007

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 26404), started 0:20:34 ago. (Use '!kill 26404' to kill it.)